In [1]:
BOT_CONFIG = {
    'intents': {
        'hello': {
            'examples': ['привет', 'добрый день', 'здравствуйте'],
            'responses': ['ну привет', 'привет юзер']
        },
        'bye': {
            'examples': ['пока'],
            'responses': ['до свидания']
        }
    },
    'failure_phrases': [
        'Ничего не знаю',
        'Моя хата с краю',
        'Ничего не понимаю'
    ]
}

In [2]:
with open('..\dialogues.txt', encoding='utf-8') as f:
    data = f.read()

In [3]:
dialogues = []
for d in data.split('\n\n'):
    replicas = []
    for t in d.split('\n')[:2]:    # первые две части строки
        t = t[2:].lower()
        replicas.append(t)
    if len(replicas) == 2 and 5<len(replicas[0])<25 and 5<len(replicas[1])<25:
        dialogues.append(replicas)    


In [4]:
BOT_DIALOGUES = dialogues[:100000]

In [5]:
CLASS_THRESHOLD = 0.9
GENER_THRESHOLD = 0.7

In [6]:
X_text = []
y = []

for intent, value in BOT_CONFIG['intents'].items():
    for example in value['examples']:
        X_text.append(example)
        y.append(intent)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.metrics.distance import edit_distance
import random


In [8]:
VC = CountVectorizer()
X =VC.fit_transform(X_text)

In [9]:
CLF = LogisticRegression()
CLF.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
def get_intent(text):
    probas = CLF.predict_proba(VC.transform([text]))
    max_value = max(probas[0])
#    print(probas[0].tolist())
#    print('get_intent', max_value)
    if max_value > CLASS_THRESHOLD:
        idx = probas[0].tolist().index(max_value)
        return CLF.classes_[idx]

In [17]:
def get_answer_by_gen(text):
    low_text = text.lower()
    for question, answer in BOT_DIALOGUES:
        if abs(len(question) - len(low_text)) / len(question) < 1 - GENER_THRESHOLD:
            dist = edit_distance(question, low_text)
            l = len(question)
            diff = dist / l
            sim = 1 - diff
#            print('get_answer_by_gen', sim)
            if sim > GENER_THRESHOLD:
                return answer
    return None

In [12]:
def get_response(intent):
    r = BOT_CONFIG['intents'][intent]['responses']
    return random.choice(r)

In [13]:
def get_failure():
    r = BOT_CONFIG['failure_phrases']
    return random.choice(r)

In [14]:
def generate_answer(text):
    
    intent = get_intent(text)
    if intent:
        response = get_response(intent)
        return response
    
    answer = get_answer_by_gen(text)
    if answer:
        return answer
    
    failure = get_failure()
    
    return failure

In [18]:
while True:
    text = input('Введите вопрос:')
    if text == 'EXIT':
        break;
    answer = generate_answer(text)
    print(answer)

Введите вопрос:здарова
как дела?
Введите вопрос:приветствую
как вы себя чувствуете?
Введите вопрос:EXIT
